In [ ]:
!pip install librosa -q

In [ ]:
import numpy as np
from scipy.signal import find_peaks
import librosa

def classify_doppler(audio_signal, fs=8000):
 # Step 1: Envelope
 envelope = np.abs(audio_signal)
 envelope = np.convolve(envelope, np.ones(500)/500, mode='same')
 
 # Step 2: Find peaks
 peaks, _ = find_peaks(envelope, height=np.mean(envelope)*2, distance=fs*0.5)
 
 if len(peaks) < 2:
 return "monophasic"
 
 # Step 3: Check reverse flow
 reverse_flow = []
 for i in range(len(peaks)-1):
 segment = envelope[peaks[i]:peaks[i+1]]
 dip = np.min(segment)
 peak_height = envelope[peaks[i]]
 if dip < 0.3 * peak_height:
 reverse_flow.append(True)
 else:
 reverse_flow.append(False)
 
 # Step 4: Classify
 if len(reverse_flow) >= 2 and all(reverse_flow):
 return "triphasic"
 elif any(reverse_flow):
 return "biphasic"
 else:
 return "monophasic"

In [ ]:
# Test na tvojim klipovima
audio1, sr1 = librosa.load('audio_dataset/poplitea_biphasic.wav', sr=8000)
print("Poplitea:", classify_doppler(audio1, sr1))

audio2, sr2 = librosa.load('audio_dataset/tibialis_posterior_monophasic.wav', sr=8000)
print("Tibialis posterior:", classify_doppler(audio2, sr2))